In [12]:
from linkedin_api import Linkedin

In [15]:
api = Linkedin('stand-with-palestine@outlook.com', 'Palestine1948')

In [178]:
list(obj['commenter'].values())[0]

{'urn': 'urn:li:member:9159810',
 'miniProfile': {'firstName': 'Gussan',
  'lastName': 'H - Opinions Are My Own',
  'dashEntityUrn': 'urn:li:fsd_profile:ACoAAACLxIIBQ2XUU8OXuTv2odBQKtxmf5_vbCY',
  'occupation': 'Digital Transformation Technology Leader',
  'objectUrn': 'urn:li:member:9159810',
  'entityUrn': 'urn:li:fs_miniProfile:ACoAAACLxIIBQ2XUU8OXuTv2odBQKtxmf5_vbCY',
  '$anti_abuse_metadata': {'$anti_abuse_uuid': '44b4ed12-94b7-41f9-b380-2693eb4f8cf9'},
  'backgroundImage': {'com.linkedin.common.VectorImage': {'artifacts': [{'width': 800,
      'fileIdentifyingUrlPathSegment': '200_800/profile-displaybackgroundimage-shrink_200_800/0/1719347651902?e=1732752000&v=beta&t=PkVSF3mwYzT9-UHrLU9nR_6yDGqeeLHbARjDXkg9LZ4',
      'expiresAt': 1732752000000,
      'height': 200},
     {'width': 1400,
      'fileIdentifyingUrlPathSegment': '350_1400/profile-displaybackgroundimage-shrink_350_1400/0/1719349913841?e=1732752000&v=beta&t=6ef7d5GrUW5y8_SeL8yWF4ST8_EQqs5LLC9ppf9Jc4k',
      'expiresA

In [3]:
import re

def extract_urn(post_url_or_urn):
    # Pattern for URN directly given as input
    urn_pattern = r'(\d+)'
    # Pattern for the first URL form (feed/update/urn:li:activity)
    pattern1 = r'urn:li:activity:(\d+)'
    # Pattern for the second URL form (activity-<ID>)
    pattern2 = r'activity-(\d+)'
    
    # Check if input is already a URN
    urn_match = re.match(urn_pattern, post_url_or_urn)
    if urn_match:
        return urn_match.group(1)
    
    # Try matching the first URL form
    urn_match = re.search(pattern1, post_url_or_urn)
    if urn_match:
        return urn_match.group(1)
    
    # Try matching the second URL form
    urn_match = re.search(pattern2, post_url_or_urn)
    if urn_match:
        return urn_match.group(1)
    
    # Return None if no URN is found
    return None

In [83]:
extract_urn("https://www.linkedin.com/feed/update/urn:li:activity:7243295716276629505") == extract_urn("https://www.linkedin.com/posts/stand-with-palestine_standwithpalestine-stopgenocide-stopisrael-activity-7243295716276629505-Qt3g?utm_source=share&utm_medium=member_desktop") == extract_urn("7243295716276629505")=="7243295716276629505"

True

In [89]:
def count_comments_and_replies(comments_data):
    total_comments = len(comments_data)  # Count the number of comments
    total_replies = 0

    # Iterate through each comment and count the number of replies
    for comment in comments_data:
        total_replies += len(comment.get('replies', []))  # Get the length of replies list, default to 0 if no replies

    return total_comments + total_replies

In [5]:
def get_comments_info(post_url, return_full_objects=False):
    try:
        # Use linked api package to get comments
        comments = api.get_post_comments(post_urn=extract_urn(post_url), comment_count=200)
    except Exception as e:
        # Often fails if no comment
        if "paginationToken" in str(e):
            print("Pagination token error. Post may have no comments")
            return ([],[]) if return_full_objects else []
        else:
            print("An error occurred:", e)
    # List of comments/replies for the post
    comments_info = []
    for i, comment in enumerate(comments):
        replies = []
        # Get replies if any first
        for reply in comments[i]['socialDetail']['comments']['elements']:
            reply_commenter_obj = reply['commenter']
            # Get the lone key inside the reply_comenter_obj which indicates user or company
            profile_attr = next(iter(reply_commenter_obj))
            # Set replier profileId
            if profile_attr == 'com.linkedin.voyager.feed.MemberActor':
                profileId = reply_commenter_obj[profile_attr]['miniProfile']['publicIdentifier']
            else:
                profileId = reply_commenter_obj[profile_attr]['miniCompany']['universalName']
                # append reply info
                replies.append({
                    "profileID": profileId,
                    "link": reply['permalink'],
                    "text": reply["commentV2"]['text'],
                }
                    )
        # Get the lone key incommener obj defining normal user or company
        commenter_obj = comment['commenter']
        profile_attr = next(iter(commenter_obj))
        # Set profileId accordingly
        if profile_attr == 'com.linkedin.voyager.feed.MemberActor':
            profileId = commenter_obj[profile_attr]['miniProfile']['publicIdentifier']
        else:
            profileId = commenter_obj[profile_attr]['miniCompany']['universalName']
        # append comment into
        comments_info.append(
            {
            "profileID": profileId,
            "link": comment['permalink'],
            "text": comment['comment']['values'][0]['value'],
            "replies": replies
            }
            )
    # comments is the full spaghetti object from the library 
    return (comments_info, comments) if return_full_objects else comments_info

# post_url = "https://www.linkedin.com/feed/update/urn:li:activity:7243337001804914688"        
# comments_info, comments_objs = get_comments_info(post_url, return_full_objects=True)

In [6]:
def get_bad_comments_links(post_url, bad_profile_ids):
    comments_info = get_comments_info(post_url)
    bad_comments_links = []
    # loop on each comment and add its link if its in bad_profile_ids
    for comment in comments_info:
        if comment['profileID'] in bad_profile_ids:
            bad_comments_links.append(comment['link'])
        # loop on each reply and add its link if its in bad_profile_ids
        for reply in comment['replies']:
            if reply['profileID'] in bad_profile_ids:
                bad_comments_links.append(reply['link'])
    return bad_comments_links

def get_bad_comments_links_many_posts(post_urls, bad_profile_ids):
    # apply get_bad_comments_links to each post and return result in one flattened list
    bad_comments_links = []
    for post_url in post_urls:
        bad_comments_links.extend(get_bad_comments_links(post_url, bad_profile_ids))
    return bad_comments_links


bad_comments_links = get_bad_comments_links_many_posts([
                                              "https://www.linkedin.com/feed/update/urn:li:activity:7243337001804914688",
                                              "https://www.linkedin.com/feed/update/urn:li:activity:7243318130352013312",
                                              "https://www.linkedin.com/feed/update/urn:li:activity:7243306804628127745",
                                             ], ["stand-with-palestine", "melek-d-700362106", "jonbarlowhudson", "dt-it-leader", "victoria-schaumburger-98944b56", "akmal-mirzaev", "nizar-alkhatib-26651057"])

In [7]:
bad_comments_links

['https://www.linkedin.com/feed/update/urn:li:ugcPost:7243337000882200576?commentUrn=urn%3Ali%3Acomment%3A%28ugcPost%3A7243337000882200576%2C7243538404557164544%29&dashCommentUrn=urn%3Ali%3Afsd_comment%3A%287243538404557164544%2Curn%3Ali%3AugcPost%3A7243337000882200576%29',
 'https://www.linkedin.com/feed/update/urn:li:ugcPost:7243337000882200576?commentUrn=urn%3Ali%3Acomment%3A%28ugcPost%3A7243337000882200576%2C7243621773353566208%29&dashCommentUrn=urn%3Ali%3Afsd_comment%3A%287243621773353566208%2Curn%3Ali%3AugcPost%3A7243337000882200576%29',
 'https://www.linkedin.com/feed/update/urn:li:ugcPost:7243337000882200576?commentUrn=urn%3Ali%3Acomment%3A%28ugcPost%3A7243337000882200576%2C7243548676856455169%29&dashCommentUrn=urn%3Ali%3Afsd_comment%3A%287243548676856455169%2Curn%3Ali%3AugcPost%3A7243337000882200576%29',
 'https://www.linkedin.com/feed/update/urn:li:ugcPost:7243337000882200576?commentUrn=urn%3Ali%3Acomment%3A%28ugcPost%3A7243337000882200576%2C7243548676856455169%29&replyUrn=ur

In [185]:
comments_info

import json

# save comments_info to a JSON file
with open('comments_info.json', 'w') as f:
    json.dump(comments_info, f, indent=4, ensure_ascii=False)


In [123]:
count_comments_and_replies(comments_info)

23

In [101]:
def find_object_index_with_substring(array_of_objects, substring):
    # Iterate over the array and check if the substring is in the 'text' attribute of each object
    for index, obj in enumerate(array_of_objects):
        if substring in obj['text']:
            return index
    # If no object is found, return None
    return None


find_object_index_with_substring(comments_info, "Oh No!!")

11

In [127]:
obj.keys()

dict_keys(['commenterProfileId', 'dashEntityUrn', 'pinned', 'edited', 'index', 'commenterForDashConversion', 'timeOffset', 'originalLanguage', 'content', 'rootSocialPermissions', 'commenter', 'socialDetail', 'urn', 'threadId', 'entityUrn', 'commentV2', 'createdTime', 'canDelete', 'comment', 'permalink', 'actions', 'contributed', 'trackingId'])

In [ ]:
https://www.linkedin.com/in/purger-elo-518265328/

In [165]:
name_obj = obj['commenter']

In [166]:
c obj['commenter']

{'com.linkedin.voyager.feed.MemberActor': {'urn': 'urn:li:member:9159810',
  'miniProfile': {'firstName': 'Gussan',
   'lastName': 'H - Opinions Are My Own',
   'dashEntityUrn': 'urn:li:fsd_profile:ACoAAACLxIIBQ2XUU8OXuTv2odBQKtxmf5_vbCY',
   'occupation': 'Digital Transformation Technology Leader',
   'objectUrn': 'urn:li:member:9159810',
   'entityUrn': 'urn:li:fs_miniProfile:ACoAAACLxIIBQ2XUU8OXuTv2odBQKtxmf5_vbCY',
   '$anti_abuse_metadata': {'$anti_abuse_uuid': '44b4ed12-94b7-41f9-b380-2693eb4f8cf9'},
   'backgroundImage': {'com.linkedin.common.VectorImage': {'artifacts': [{'width': 800,
       'fileIdentifyingUrlPathSegment': '200_800/profile-displaybackgroundimage-shrink_200_800/0/1719347651902?e=1732752000&v=beta&t=PkVSF3mwYzT9-UHrLU9nR_6yDGqeeLHbARjDXkg9LZ4',
       'expiresAt': 1732752000000,
       'height': 200},
      {'width': 1400,
       'fileIdentifyingUrlPathSegment': '350_1400/profile-displaybackgroundimage-shrink_350_1400/0/1719349913841?e=1732752000&v=beta&t=6ef7d5

In [140]:
name_obj = obj['commenter']['com.linkedin.voyager.feed.MemberActor']['miniProfile']['publicIdentifier']


'dt-it-leader'

In [139]:
obj = comments_objs[1]

# save json
with open('comments_objs.json', 'w') as f:
    json.dump(obj, f, indent=4, ensure_ascii=False)

In [172]:
c = comments_objs[0]['socialDetail']['comments']['elements'][0]['commenter']
# save c as json:
with open('c.json', 'w') as f:
    json.dump(c[next(iter(c))], f, indent=4, ensure_ascii=False)

In [85]:
comments_info

# get the index of the object having Oh No!! in text
comments_info[0]['text'].find("Oh No!!")

[{'link': 'https://www.linkedin.com/feed/update/urn:li:ugcPost:7243295715513303041?commentUrn=urn%3Ali%3Acomment%3A%28ugcPost%3A7243295715513303041%2C7243378888683257856%29&dashCommentUrn=urn%3Ali%3Afsd_comment%3A%287243378888683257856%2Curn%3Ali%3AugcPost%3A7243295715513303041%29',
  'text': 'Imagine this was happening to people in the US. For a YEAR straight.\n\nPalestinians have been facing doomsday level horrors every day for the past year. It can all stop immediately if the US enforces an arms embargo on Israel.',
  'replies': [{'link': 'https://www.linkedin.com/feed/update/urn:li:ugcPost:7243295715513303041?commentUrn=urn%3Ali%3Acomment%3A%28ugcPost%3A7243295715513303041%2C7243378888683257856%29&replyUrn=urn%3Ali%3Acomment%3A%28ugcPost%3A7243295715513303041%2C7243594235067719680%29&dashCommentUrn=urn%3Ali%3Afsd_comment%3A%287243378888683257856%2Curn%3Ali%3AugcPost%3A7243295715513303041%29&dashReplyUrn=urn%3Ali%3Afsd_comment%3A%287243594235067719680%2Curn%3Ali%3AugcPost%3A72432957

In [29]:
len(comment_links)

0

In [93]:
c=0 # comment number
comments[c]['comment']['values'][0]['value']

'And still they use mass destruction of babies \n\nWhy is there no CEASEFIRE .? Children are \n\nContinuing to be immunised for Polio 👐🏾🤲🏼\n\nPalestine 🇵🇸🩸is bleeding 🩸 stop bombs 💣 \n\nI stand with Palestine 🇵🇸 freedom peace ☮️\n\nWake up world 🌎 ASAP NOW SOS 🆘'

In [90]:
i=0 # reply number
comments[0]['socialDetail']['comments']['elements'][i]["commentV2"]['text']

'Anne Reeve We were never able to understand this. Thank you so much for your voice.\n'

In [89]:
i=1
comments[0]['socialDetail']['comments']['elements'][i]['comment']['values'][1]['value']

' Thank you, SWP.'

In [20]:
print(comments[0])

{'commenterProfileId': 'ACoAAAgEc6ABRkky1hofUfSkj2OJATmTBUrlwdo', 'dashEntityUrn': 'urn:li:fsd_comment:(7240329572053876736,urn:li:ugcPost:7240079285212827648)', 'pinned': False, 'edited': False, 'index': -1, 'commenterForDashConversion': {'urn': 'urn:li:member:134509472', 'image': {'attributes': [{'sourceType': 'PROFILE_PICTURE', 'miniProfile': {'firstName': 'Or', 'lastName': 'Lapid', 'dashEntityUrn': 'urn:li:fsd_profile:ACoAAAgEc6ABRkky1hofUfSkj2OJATmTBUrlwdo', 'occupation': 'Snr. Field Apps Engineer at Micron Technology', 'objectUrn': 'urn:li:member:134509472', 'entityUrn': 'urn:li:fs_miniProfile:ACoAAAgEc6ABRkky1hofUfSkj2OJATmTBUrlwdo', '$anti_abuse_metadata': {'$anti_abuse_uuid': '4aca8edc-eba3-446d-aea8-02d33acf71d3'}, 'backgroundImage': {'com.linkedin.common.VectorImage': {'artifacts': [{'width': 800, 'fileIdentifyingUrlPathSegment': '200_800/profile-displaybackgroundimage-shrink_200_800/0/1698991294976?e=1731542400&v=beta&t=Q0EljcZZvFBK7J86IQ1hdLqP5ArtsfI6xcXsVcmtS3E', 'expires